# Import Library

In [ ]:
# !pip install -q tensorflow-ranking
!pip install -q tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,754 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import io
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify
# from feature import fitur1_ML, fitur2_ml
import os
import random
import pytesseract
from PIL import Image
# from fitur1_ML import predict as predict_books_1#
# from fitur2_ml import load_and_predict_genre, recommend_books, extract_text_from_image

In [ ]:
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import tensorflow_recommenders as tfrs

# Import Dataset and Model

In [ ]:
capstone_path = kagglehub.dataset_download("dikiiwahyudi/capstone-literify")
print('Data source import complete')

100%|██████████| 37.6M/37.6M [00:00<00:00, 62.1MB/s]

Extracting files...


Data source import complete


In [ ]:
# Make folder 'data'
data_folder = 'data'
os.makedirs(data_folder, exist_ok=True)
shutil.move(capstone_path, os.path.join(data_folder, 'full-data'))
print(f'Dataset has been moved to folder {data_folder}')

In [ ]:
# Make folder 'model'
model_folder = 'model'
os.makedirs(model_folder, exist_ok=True)

In [ ]:
!wget -P /content/data https://raw.githubusercontent.com/Literify/ML/refs/heads/main/data/content_df.csv
!wget -P /content/data https://raw.githubusercontent.com/Literify/ML/refs/heads/main/data/user_ids.csv
!wget -P /content/model https://raw.githubusercontent.com/Literify/ML/refs/heads/main/model/cosine_similarity.pkl
!wget -P /content/model https://raw.githubusercontent.com/Literify/ML/refs/heads/main/model/label_encoder.pkl
!wget -P /content/model https://raw.githubusercontent.com/Literify/ML/refs/heads/main/model/vectorizer_vocab.pkl
!wget -P /content/model https://raw.githubusercontent.com/Literify/ML/refs/heads/main/model/model_genre_classification_weights.h5
!wget -P /content/model https://raw.githubusercontent.com/Literify/ML/refs/heads/main/model/model_recomendation_weights.h5

--2024-12-10 01:08:00--  https://raw.githubusercontent.com/Literify/ML/refs/heads/main/data/content_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23594053 (23M) [text/plain]
Saving to: ‘/content/data/content_df.csv’

content_df.csv      100%[===================>]  22.50M  --.-KB/s    in 0.09s   

2024-12-10 01:08:00 (237 MB/s) - ‘/content/data/content_df.csv’ saved [23594053/23594053]

--2024-12-10 01:08:01--  https://raw.githubusercontent.com/Literify/ML/refs/heads/main/data/user_ids.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

# Load Data and Model

In [ ]:
sampled_categories = pd.read_csv('/content/data/full-data/sampled_categories.csv')
content_df = pd.read_csv('/content/data/content_df.csv')
user_ids_df = pd.read_csv('/content/data/user_ids.csv')

# Fitur 1: Search Bar

In [ ]:
# Function to display catalog-style recommendations
def display_catalog(recommendations, top_n=3):
    display_str = f"<h3>Top {top_n} Recommendations:</h3><br>"

    for idx, row in recommendations.iterrows():
        display_str += f"""
            <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px;">
                <h4>{row['book_title']}</h4>
                <p><strong>Authors:</strong> {row['authors']}</p>
                <p><strong>Genre:</strong> {row['genre']}</p>
                <p><strong>Publisher:</strong> {row['publisher']}</p>
                <p><strong>Price:</strong> {row['Price']}</p>
                <p><strong>Description:</strong> {row['description']}</p>
                <img src="{row['image']}" alt="{row['book_title']}" width="100" height="150" style="display:block; margin-top: 10px;">
                <a href="{row['previewLink']}" target="_blank">Preview</a> |
                <a href="{row['infoLink']}" target="_blank">More Info</a>
            </div>
        """

    # Display the catalog-style information
    display(HTML(display_str))

In [ ]:
def predict(title, data, cos_sim, similarity_weight=0.7, top_n=10):
    index_movie = data[data['book_title'] == title].index
    similarity = cos_sim[index_movie].T

    sim_df = pd.DataFrame(similarity, columns=['similarity'])
    final_df = pd.concat([data, sim_df], axis=1)

    final_df['final_score'] = final_df['weighted_average']*(1-similarity_weight) + final_df['similarity']*similarity_weight

    final_df_sorted = final_df.sort_values(by='final_score', ascending=False).head(top_n)
    final_df_sorted_show = final_df_sorted[['book_title', 'description', 'authors', 'genre', 'publisher', 'Price', 'image', 'previewLink', 'infoLink']]

    selected_title = final_df[final_df['book_title'] == title][['book_title', 'description', 'authors', 'genre', 'publisher', 'Price', 'image', 'previewLink', 'infoLink']].to_dict(orient='records')[0]

    recommendations = final_df_sorted_show.to_dict(orient='records')

    output = {
        "selected_title": selected_title,
        "recommendations": recommendations
    }

    return output

In [ ]:
# Load the cosine similarity matrix from pickle
cos_sim = pickle.load(open('/content/model/cosine_similarity.pkl', 'rb'))
# Convert the cosine similarity matrix to a dense format
cos_sim_dense = cos_sim.toarray()

In [ ]:
predict("It's Not All Song and Dance: A Life Behind the Scenes in the Performing Arts", content_df, cos_sim_dense, similarity_weight=0.7, top_n=3)

{'selected_title': {'book_title': "It's Not All Song and Dance: A Life Behind the Scenes in the Performing Arts",
  'description': '"Finally, Gershunoff\'s memoir reveals the fruits of his distinguished career in the performing arts, providing valuable lessons for today\'s performing arts managers and presenters, as well as aspiring artists."--BOOK JACKET.',
  'authors': 'Maxim Gershunoff, Leon Van Dyke',
  'genre': 'Biography & Autobiography',
  'publisher': 'Hal Leonard Corporation',
  'Price': 24.95,
  'image': 'http://books.google.com/books/content?id=nosIAQAAMAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api',
  'previewLink': 'http://books.google.nl/books?id=nosIAQAAMAAJ&q=It%27s+Not+All+Song+and+Dance:+A+Life+Behind+the+Scenes+in+the+Performing+Arts&dq=It%27s+Not+All+Song+and+Dance:+A+Life+Behind+the+Scenes+in+the+Performing+Arts&hl=&cd=1&source=gbs_api',
  'infoLink': 'http://books.google.nl/books?id=nosIAQAAMAAJ&dq=It%27s+Not+All+Song+and+Dance:+A+Life+Behind+the+Scenes+in+t

In [ ]:
# Create a dropdown widget for book titles
dropdown = widgets.Dropdown(
    options=content_df['book_title'].tolist(),
    description='Book Title:',
    disabled=False
)

# Function to display top n recommendations when a book is selected
def on_select(change):
    selected_title = change.new
    recommendations_html = predict(selected_title, content_df, cos_sim_dense, similarity_weight=0.7, top_n=3)

    # Display the catalog-style information
    display(HTML(recommendations_html))

# Attach the function to the dropdown widget
dropdown.observe(on_select, names='value')

# Display the dropdown widget
display(dropdown)

Dropdown(description='Book Title:', options=("It's Not All Song and Dance: A Life Behind the Scenes in the Per…

# Fitur 2: Predict Book Genre and Give Recomendation

## Text Classification

In [ ]:
import tensorflow as tf
import pickle
import numpy as np

def load_and_predict_genre(text, model_weights_path, vectorizer_vocab_path, label_encoder_path):
    # Parameters
    VOCAB_SIZE = 10000
    MAX_LENGTH = 32
    PADDING_TYPE = 'pre'
    TRUNC_TYPE = 'post'
    EMBEDDING_DIM = 16
    LSTM_DIM = 32
    DENSE_DIM = 24

    # Model definition
    model_lstm = tf.keras.Sequential([
        tf.keras.Input(shape=(MAX_LENGTH,)),
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_DIM)),
        tf.keras.layers.Dense(DENSE_DIM, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Load the model weights
    model_lstm.load_weights(model_weights_path)

    # Load the label encoder
    with open(label_encoder_path, 'rb') as file:
        label_encoder = pickle.load(file)

    # Load the saved vocabulary
    with open(vectorizer_vocab_path, 'rb') as file:
        vocabulary = pickle.load(file)

    # Recreate the TextVectorization layer using the loaded vocabulary
    vectorize_layer = tf.keras.layers.TextVectorization(vocabulary=vocabulary)

    # Remove stopwords (helper function)
    def remove_stopwords(sentence):
        stopwords = [
            "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
            "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
            "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
            "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how",
            "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's",
            "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our",
            "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some",
            "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
            "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under",
            "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when",
            "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would",
            "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"
        ]
        sentence = sentence.lower()
        word_list = sentence.split()
        words = [w for w in word_list if w not in stopwords]
        return " ".join(words)

    # Preprocess and predict genre
    processed_text = remove_stopwords(text)
    vectorized_text = vectorize_layer([processed_text])
    padded_text = tf.keras.preprocessing.sequence.pad_sequences(
        vectorized_text, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNC_TYPE
    )
    genre_probabilities = model_lstm.predict(padded_text)
    predicted_genre_index = np.argmax(genre_probabilities, axis=1)[0]
    predicted_genre = label_encoder.inverse_transform([predicted_genre_index])[0]

    return predicted_genre

In [ ]:
load_and_predict_genre("It's Not All Song and Dance: A Life Behind the Scenes in the Performing Arts",
                       model_weights_path="/content/model/model_genre_classification_weights.h5",
                       vectorizer_vocab_path="/content/model/vectorizer_vocab.pkl",
                       label_encoder_path="/content/model//label_encoder.pkl"
)

1/1 [==============================] - 13s 13s/step


'Biography & Autobiography'

## Recomendation System

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np

def predict_book_recommendation(user, filtered_books_df, unique_book_titles, unique_user_ids, model_weights_path, top_n=3):
    # Load and preprocess data
    books = tf.data.Dataset.from_tensor_slices(dict(filtered_books_df[['book_title']]))
    books = books.map(lambda x: x["book_title"])

    # Define the recommendation model
    class BookModel(tfrs.models.Model):
        def __init__(self, rating_weight: float, retrieval_weight: float, unique_book_titles, unique_user_ids) -> None:
            super().__init__()
            embedding_dimension = 32

            self.book_model: tf.keras.layers.Layer = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=unique_book_titles, mask_token=None),
                tf.keras.layers.Embedding(len(unique_book_titles) + 1, embedding_dimension)
            ])
            self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
            ])
            self.rating_model = tf.keras.Sequential([
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dense(1),
            ])
            self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
            )
            self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
                metrics=tfrs.metrics.FactorizedTopK(
                    candidates=books.batch(248).map(self.book_model)
                )
            )
            self.rating_weight = rating_weight
            self.retrieval_weight = retrieval_weight

        def call(self, features: dict) -> tf.Tensor:
            user_embeddings = self.user_model(features["user_id"])
            book_embeddings = self.book_model(features["book_title"])
            return (
                user_embeddings,
                book_embeddings,
                self.rating_model(tf.concat([user_embeddings, book_embeddings], axis=1)),
            )

        def compute_loss(self, features: dict, training=False) -> tf.Tensor:
            ratings = features.pop("rating")
            user_embeddings, book_embeddings, rating_predictions = self(features)
            rating_loss = self.rating_task(labels=ratings, predictions=rating_predictions)
            retrieval_loss = self.retrieval_task(user_embeddings, book_embeddings)
            return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

    # Initialize the model
    model = BookModel(rating_weight=1.0, retrieval_weight=1.0,
                      unique_book_titles=unique_book_titles, unique_user_ids=unique_user_ids)

    # Build the model with dummy data
    dummy_features = {
        "user_id": tf.constant([unique_user_ids[0]]),
        "book_title": tf.constant([unique_book_titles[0]]),
        "rating": tf.constant([1.0]),
    }
    _ = model(dummy_features)  # This builds the model

    # Load pre-trained weights
    model.load_weights(model_weights_path)

    # Build index for retrieval
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
        tf.data.Dataset.zip((books.batch(100), books.batch(100).map(model.book_model)))
    )

    # Generate recommendations
    _, titles = index(tf.constant([str(user)]))
    recommended_titles = [title.decode("utf-8") for title in titles[0, :top_n].numpy()]

    # Filter and return details of the recommended books
    recommendations = filtered_books_df[filtered_books_df['book_title'].isin(recommended_titles)]
    return recommendations.to_dict(orient="records")

In [ ]:
sampled_categories = sampled_categories.rename(columns={'review/score': 'user_rating', 'User_id': 'user_id', 'Title': 'book_title'})

In [ ]:
unique_books_df = sampled_categories[['book_title']].drop_duplicates()

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(dict(sampled_categories[['user_id', 'book_title', 'user_rating']]))
books = tf.data.Dataset.from_tensor_slices(dict(unique_books_df[['book_title']]))

In [ ]:
ratings = ratings.map(lambda x: {
    "book_title": x["book_title"],
    "user_id": x["user_id"],
    "rating": float(x["user_rating"])
})

books = books.map(lambda x: x["book_title"])

In [ ]:
book_titles = books.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

unique_book_titles = np.unique(np.concatenate(list(book_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
# Example usage
recommendations = predict_book_recommendation(
     user="user_id_123",
     filtered_books_df=content_df,
     unique_book_titles=unique_book_titles,
     unique_user_ids=unique_user_ids,
     model_weights_path='/content/model/model_recomendation_weights.h5',
     top_n=3
)

In [ ]:
recommendations

[{'book_title': '500 Low-Carb Recipes: 500 Recipes from Snacks to Dessert, That the Whole Family Will Love',
  'description': 'Low-fat or low-carb? A recent New York Times Magazine (July 7, 2002) cover story answered this question and said that Dr. Atkins was right all along, "its not fat that makes us fat but carbohydrates." Though the government has spent hundreds of millions of dollars in research trying to prove that fat is the cause of obesity, there has been a subtle shift in the scientific consensus over the past five years supporting what the low-carb diet doctors have been saying all along: if we eat less carbohydrates, we will lose weight and live longer. One of the toughest challenges of any diet is having enough variety and choices to keep the dieter from losing interest. The most common reason that people abandon their diet is boredom but 500 LOW CARB RECIPES: 500 Recipes, From Snacks to Dessert, That the Whole Family Will Love by Dana Carpender has more than enough recipe

In [ ]:
# Function to recommend books based on user input
def recommend_books(user_ids_df, content_df, predicted_genre, unique_book_titles, unique_user_ids, model_weights_path, top_n=3, choice="only_genre"):
    user = random.choice(user_ids_df['user_id'].tolist())
    if choice == "only_genre":
        # Filter books only by the predicted genre
        content_df = content_df[content_df['genre'] == predicted_genre].drop_duplicates()
        recommendations = predict_book_recommendation(user=user, filtered_books_df=content_df, unique_book_titles=unique_book_titles,
                                                      unique_user_ids=unique_user_ids, model_weights_path=model_weights_path,
                                                      top_n=3)
        return recommendations
    elif choice == "all_genre":
        # Recommend books from all genres, including the predicted one
        content_df = content_df.drop_duplicates()
        recommendations = predict_book_recommendation(user=user, filtered_books_df=content_df, unique_book_titles=unique_book_titles,
                                                      unique_user_ids=unique_user_ids, model_weights_path=model_weights_path,
                                                      top_n=3)
        return recommendations
    elif choice == "no_genre":
        print("No recommendations will be provided. Thank you!")
        return None

In [ ]:
TEST = recommend_books(user_ids_df, content_df, "Biography & Autobiography", unique_book_titles, unique_user_ids, model_weights_path='/content/model/model_recomendation_weights.h5', top_n=3, choice="only_genre")

In [ ]:
print(TEST)

[{'book_title': 'Left to Tell: Discovering God Amidst The Rwandan Holocaust', 'description': 'Immaculee Ilibagiza grew up in a country she loved, surrounded by a family she cherished. But in 1994 her idyllic world was ripped apart as Rwanda descended into a bloody genocide. Immaculee’s family was brutally murdered during a killing spree that lasted three months and claimed the lives of nearly a million Rwandans. Incredibly, Immaculee survived the slaughter. For 91 days, she and seven other women huddled silently together in the cramped bathroom of a local pastor while hundreds of machete-wielding killers hunted for them. It was during those endless hours of unspeakable terror that Immaculee discovered the power of prayer, eventually shedding her fear of death and forging a profound and lasting relationship with God. She emerged from her bathroom hideout having discovered the meaning of truly unconditional love—a love so strong she was able seek out and forgive her family’s killers. The

## OCR

In [ ]:
from PIL import Image
import pytesseract

def extract_text_from_image(image_path):
        # Open the image
        img = Image.open(image_path)
        # Extract text using Tesseract
        extracted_text = pytesseract.image_to_string(img)

        # Clean the extracted text
        preprocessed_text = ' '.join(extracted_text.split())

        # Display the extracted text
        # print("Extracted text from the image:")
        # print(preprocessed_text)
        return preprocessed_text

## Full Function 1

In [ ]:
# Example usage
image_path = '/content/book-covers-big-2019101610.jpg' # Please input your image path
title_ocr = extract_text_from_image(image_path)
genre = load_and_predict_genre(title_ocr,
                               model_weights_path="/content/model/model_genre_classification_weights.h5",
                               vectorizer_vocab_path="/content/model/vectorizer_vocab.pkl",
                               label_encoder_path="/content/model//label_encoder.pkl")
genre

1/1 [==============================] - 1s 1s/step


'Self-Help'

## Full Function 2

In [ ]:
# Example usage
choice = "only_genre"
TEST = recommend_books(user_ids_df, content_df, genre, unique_book_titles, unique_user_ids, model_weights_path='/content/model/model_recomendation_weights.h5', top_n=3, choice="only_genre")

In [ ]:
TEST

[{'book_title': 'Indigo Dreams: Adult Relaxation-Guided Meditation/Relaxation Techniques decrease anxiety, stress, anger',
  'description': 'Children love to unwind and relax with this fun exercise known as "muscular relaxation". Children relate to the angry octopus in this story as the sea child shows him how to take a deep breath, calm down, and manage his anger. This effective stress and anger management technique focuses awareness on various muscle groups to create a complete resting of the mind and body. Muscular relaxation can lower stress and anxiety levels. It can be used to decrease pain and anger. This engaging story quiets the mind and relaxes the body so your child can let go of anger and fall asleep peacefully. This is one of four stories featured on the Indigo Ocean Dreams CD.',
  'authors': 'Lori Lite',
  'genre': 'Self-Help',
  'publisher': 'Stress Free Kids',
  'Price': 10.85,
  'image': 'http://books.google.com/books/content?id=7SA_SPBiCYUC&printsec=frontcover&img=1&z

## Raw

In [ ]:
class BookModel(tfrs.models.Model):
    def __init__(self, rating_weight: float, retrieval_weight: float,
                 unique_book_titles, unique_user_ids) -> None:
        super().__init__()

        embedding_dimension = 32

        self.book_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_book_titles, mask_token=None),
            tf.keras.layers.Embedding(len(unique_book_titles) + 1, embedding_dimension)
        ])
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=books.batch(248).map(self.book_model)
            )
        )
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: dict) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        book_embeddings = self.book_model(features["book_title"])
        return (
            user_embeddings,
            book_embeddings,
            self.rating_model(tf.concat([user_embeddings, book_embeddings], axis=1)),
        )

    def compute_loss(self, features: dict, training=False) -> tf.Tensor:
        ratings = features.pop("rating")
        user_embeddings, book_embeddings, rating_predictions = self(features)
        rating_loss = self.rating_task(labels=ratings, predictions=rating_predictions)
        retrieval_loss = self.retrieval_task(user_embeddings, book_embeddings)
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

In [ ]:
model = BookModel(rating_weight=1.0, retrieval_weight=1.0,
                  unique_book_titles=unique_book_titles, unique_user_ids=unique_user_ids)

In [ ]:
# Build the model with dummy data
dummy_features = {
    "user_id": tf.constant([unique_user_ids[0]]),
    "book_title": tf.constant([unique_book_titles[0]]),
    "rating": tf.constant([1.0]),
}
_ = model(dummy_features)  # This builds the model

In [ ]:
model.load_weights('/content/model/model_recomendation_weights.h5')

In [ ]:
def predict_book_recomendation(user, filtered_books_df, top_n=3):
    books = tf.data.Dataset.from_tensor_slices(dict(filtered_books_df[['book_title']]))
    books = books.map(lambda x: x["book_title"])

    # Create a model that takes in raw query features
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    # Recommends books out of the entire books dataset
    index.index_from_dataset(
        tf.data.Dataset.zip((books.batch(100), books.batch(100).map(model.book_model)))
    )

    # Get recommendations
    _, titles = index(tf.constant([str(user)]))
    recommended_titles = [title.decode("utf-8") for title in titles[0, :top_n].numpy()]

    # Filter details from the input DataFrame
    recommendations = filtered_books_df[filtered_books_df['book_title'].isin(recommended_titles)]
    return display_catalog(recommendations, top_n)
def predict_rating(user, book):
    trained_book_embeddings, trained_user_embeddings, predicted_rating = model({
          "user_id": np.array([str(user)]),
          "book_title": np.array([book])
      })
    print("Predicted rating for {}: {}".format(book, predicted_rating.numpy()[0][0]))

## OCR

In [ ]:
def extract_text_from_image(image_path):
        # Open the image
        img = Image.open(image_path)
        # Extract text using Tesseract
        extracted_text = pytesseract.image_to_string(img)

        # Clean the extracted text
        preprocessed_text = ' '.join(extracted_text.split())

        # Display the extracted text
        print("Extracted text from the image:")
        print(preprocessed_text)
        return preprocessed_text

## Full Function

In [ ]:
# Function to recommend books based on user input
def recommend_books(user_ids_df, content_df, predicted_genre, top_n=3, choice="only_genre"):
    user = random.choice(user_ids_df['user_id'].tolist())
    if choice == "only_genre":
        # Filter books only by the predicted genre
        content_df = content_df[content_df['genre'] == predicted_genre].drop_duplicates()
        predict_book_recomendation(user, content_df, top_n=3)
    elif choice == "all_genre":
        # Recommend books from all genres, including the predicted one
        content_df = content_df.drop_duplicates()
        predict_book_recomendation(user, content_df, top_n=3)
    elif choice == "no_genre":
        print("No recommendations will be provided. Thank you!")
    else:
        print("Invalid choice. No recommendations provided.")

# Full workflow to extract text, predict genre, and recommend books
def extract_predict_recommend(title_ocr, user_ids_df, content_df, top_n=3, choice="only_genre"):
    predicted_genre = predict_genre_book(title_ocr)
    if predicted_genre:
      print(f"\nPredicted Genre: {predicted_genre}")
      recommend_books(user_ids_df, content_df, predicted_genre, top_n, choice)
    else:
      print("Failed to predict the genre.")

In [ ]:
import requests
from PIL import Image
import pytesseract
from io import BytesIO

def extract_text_from_image_url(image_url):
    """
    Extracts text from an image available at a URL using Tesseract OCR.

    Args:
        image_url (str): The URL of the image to process.

    Returns:
        str: The extracted and cleaned text from the image.
    """
    # Download the image from the URL
    response = requests.get(image_url)
    if response.status_code == 200:
        # Open the image from the downloaded content
        img = Image.open(BytesIO(response.content))
        # Extract text using Tesseract
        extracted_text = pytesseract.image_to_string(img)

        # Clean the extracted text
        preprocessed_text = ' '.join(extracted_text.split())

        # Display the extracted text
        print("Extracted text from the image:")
        print(preprocessed_text)
        return preprocessed_text
    else:
        print(f"Failed to fetch the image. Status code: {response.status_code}")
        return None

# Example usage
image_url = "https://storage.googleapis.com/literify/IMG20241202141307.jpg"
text = extract_text_from_image_url(image_url)


Extracted text from the image:



In [ ]:
# Example usage
image_path = 'https://storage.googleapis.com/literify/IMG20241202141307.jpg' # Please input your image path
title_ocr = extract_text_from_image(image_path)
#choice = "only_genre"
#extract_predict_recommend(title_ocr, user_ids_df, content_df, top_n=3, choice=choice)

FileNotFoundError: [Errno 2] No such file or directory: '/content/https:/storage.googleapis.com/literify/IMG20241202141307.jpg'